In [1]:
# Dependencies
import requests
from config import api_key
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager


In [2]:
url = 'https://www.in2013dollars.com/us/inflation/1930?amount=1'

In [3]:
cpi_inflation = pd.read_html(url)
cpi_inflation

[                            0               1
 0     Cumulative price change       1,486.09%
 1      Average inflation rate           3.08%
 2  Converted amount ($1 base)          $15.86
 3  Price difference ($1 base)          $14.86
 4                 CPI in 1930          16.700
 5                 CPI in 2021         264.877
 6           Inflation in 1930          -2.34%
 7           Inflation in 2021           2.62%
 8                  $1 in 1930  $15.86 in 2021,
     Year Dollar Value Inflation Rate
 0   1930        $1.00         -2.34%
 1   1931        $0.91         -8.98%
 2   1932        $0.82         -9.87%
 3   1933        $0.78         -5.11%
 4   1934        $0.80          3.08%
 ..   ...          ...            ...
 87  2017       $14.68          2.13%
 88  2018       $15.04          2.49%
 89  2019       $15.31          1.76%
 90  2020       $15.50          1.23%
 91  2021       $15.86         2.34%*
 
 [92 rows x 3 columns],
                  Initial value              Eq

In [4]:
cpi_inflation_df = cpi_inflation[1]
cpi_inflation_df

,Year,Dollar Value,Inflation Rate
0,1930,$1.00,-2.34%
1,1931,$0.91,-8.98%
2,1932,$0.82,-9.87%
3,1933,$0.78,-5.11%
4,1934,$0.80,3.08%
...,...,...,...
87,2017,$14.68,2.13%
88,2018,$15.04,2.49%
89,2019,$15.31,1.76%
90,2020,$15.50,1.23%


In [ ]:
url = 'https://alfred.stlouisfed.org/series/downloaddata?seid=MABMM301USA657S'
realtime_start = '1990-01-01'
realtime_end = '2020-12-31'

In [ ]:
# https://api.stlouisfed.org/fred/series/observations?series_id=GNPCA
# &realtime_start=1776-07-04&realtime_end=9999-12-31&api_key=abcdefghijklmnopqrstuvwxyz123456
# Build query URL
query_url = url + "&realtime_start=" + realtime_start + "&realtime_end=" + realtime_end + "&api_key=" + api_key + "&file_type=json"
query_url

In [ ]:
response = requests.get(query_url)
response

In [ ]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

In [ ]:
url = 'https://alfred.stlouisfed.org/series/downloaddata?seid=MABMM301USA657S&realtime_start=1990-01-01&realtime_end=2020-12-31&api_key=d282d9b79098a2dd1a31f63af87c22ab&file_type=json'
browser.visit(url)

In [ ]:
for x in range(1, 2):

    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')

    tables = soup.find('button', id = 'form_download_data')

    for table in tables:
        print(table)

    

In [ ]:
m3_data = pd.read_excel('US-M3.xls')
m3_data